[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/pixeltable/pixeltable/blob/master/docs/release/tutorials/rag-operations.ipynb)&nbsp;&nbsp;
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pixeltable/pixeltable/blob/master/docs/release/tutorials/rag-operations.ipynb)

# RAG Operations in Pixeltable

In this tutorial, we'll explore Pixeltable's flexible handling of RAG operations on unstructured text. In a traditional AI workflow, such operations might be implemented as a Python script that runs on a periodic schedule or in response to certain events. In Pixeltable, as with everything else, they are implemented as persistent table operations that update incrementally as new data becomes available. In our tutorial workflow, we'll chunk Wikipedia articles in various ways with a document splitter, then apply several kinds of embeddings to the chunks.

## Set Up the Table Structure

We start by installing the necessary dependencies, creating a Pixeltable directory `rag_ops_demo` (if it doesn't already exist), and setting up the table structure for our new workflow.

In [ ]:
%pip install -q sentence-transformers pixeltable

In [1]:
import pixeltable as pxt

# Create the Pixeltable workspace
pxt.create_dir('rag_ops_demo', ignore_errors=True)

# Clean the database to ensure we're using fresh table instances
# (in case this demo has been run before)
pxt.drop_table('rag_ops_demo.short_char_chunks', ignore_errors=True)
pxt.drop_table('rag_ops_demo.short_chunks', ignore_errors=True)
pxt.drop_table('rag_ops_demo.chunks', ignore_errors=True)
pxt.drop_table('rag_ops_demo.sentences', ignore_errors=True)
pxt.drop_table('rag_ops_demo.docs', ignore_errors=True)

Connected to Pixeltable database at: postgresql://postgres:@/pixeltable?host=/Users/asiegel/.pixeltable/pgdata
Created directory `rag_ops_demo`.


## Creating Tables and Views

Now we'll create the tables that represent our workflow, starting with a table to hold references to source documents. The table contains a single column `source_doc` whose elements have type `pxt.DocumentType`, representing a general document instance. In this tutorial, we'll be working with HTML documents, but Pixeltable supports a range of other document types, such as Markdown and PDF.

In [2]:
docs = pxt.create_table('rag_ops_demo.docs', {'source_doc': pxt.DocumentType()})

Created table `docs`.


If we take a peek at the `docs` table, we see its very simple structure.

In [3]:
docs

Column Name,Type,Computed With
source_doc,document,


Next we create a view to represent chunks of our HTML documents. A Pixeltable view is a virtual table, which is dynamically derived from a source table by applying a transformation and/or selecting a subset of data. In this case, our view represents a one-to-many transformation from source documents into individual sentences. This is achieved using Pixeltable's built-in `DocumentSplitter` class.

Note that the `docs` table is currently empty, so creating this view doesn't actually *do* anything yet: it simply defines an operation that we want Pixeltable to execute when it sees new data.

In [4]:
from pixeltable.iterators.document import DocumentSplitter

sentences = pxt.create_view(
    'rag_ops_demo.sentences',  # Name of the view
    docs,  # Table from which the view is derived
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='sentence',  # Chunk docs into sentences
        metadata='title,heading,sourceline'
    )
)

Created view `sentences` with 0 rows, 0 exceptions.


Let's take a peek at the new `sentences` view.

In [5]:
sentences

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
heading,json,
sourceline,int,
source_doc,document,


We see that `sentences` inherits the `source_doc` column from `docs`, together with some new fields:
- `pos`: The position in the source document where the sentence appears.
-  `text`: The text of the sentence.
- `title`, `heading`, and `sourceline`: The metadata we requested when we set up the view.

## Data Ingestion

Ok, now it's time to insert some data into our workflow. A document in Pixeltable is just a URL; the following command inserts a single row into the `docs` table with the `source_doc` field set to the specified URL:

In [6]:
docs.insert(source_doc='https://en.wikipedia.org/wiki/Marc_Chagall')

Inserting rows into `docs`: 1 rows [00:00, 417.34 rows/s]
Inserting rows into `sentences`: 1461 rows [00:00, 3707.91 rows/s]
Inserted 1462 rows with 0 errors.


UpdateStatus(num_rows=1462, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

We can see that two things happened. First, a single row was inserted into `docs`, containing the URL representing our source document. Then, the view `sentences` was incrementally updated by applying the `DocumentSplitter` according to the definition of the view. This illustrates an important principle in Pixeltable: by default, anytime Pixeltable sees new data, the update is incrementally propagated to any downstream views or computed columns.

We can see the effect of the insertion with the `select` command. There's a single row in `docs`:

In [7]:
docs.select(docs.source_doc.fileurl).show()

source_doc_fileurl
https://en.wikipedia.org/wiki/Marc_Chagall


And here are the first 20 rows in `sentences`. The content of the article is broken into individual sentences, as expected.

In [8]:
sentences.select(sentences.text, sentences.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية,{'1': 'Marc Chagall'}
Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी,{'1': 'Marc Chagall'}
Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย,{'1': 'Marc Chagall'}
Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש,{'1': 'Marc Chagall'}
"粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here.",{'1': 'Marc Chagall'}
"For other uses, see Chagall (disambiguation) .",{'1': 'Marc Chagall'}
"Marc Chagall Chagall, c. 1920",{'1': 'Marc Chagall'}
"Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus)",{'1': 'Marc Chagall'}
[1] Died 28 March 1985 (1985-03-28) (aged 97),{'1': 'Marc Chagall'}


## Experimenting with Chunking

Of course, chunking into sentences isn't the only way to split a document. Perhaps we want to experiment with different chunking methodologies, in order to see which one performs best in a particular application. Pixeltable makes it easy to do this, by creating several views of the same source table. Here are a few examples. Notice that as each new view is created, it is initially populated from the data already in `docs`.

In [9]:
chunks = pxt.create_view(
    'rag_ops_demo.chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,token_limit',
        limit=2048,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `chunks`: 205 rows [00:00, 19755.36 rows/s]
Created view `chunks` with 205 rows, 0 exceptions.


In [10]:
short_chunks = pxt.create_view(
    'rag_ops_demo.short_chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,token_limit',
        limit=72,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `short_chunks`: 531 rows [00:00, 25065.28 rows/s]
Created view `short_chunks` with 531 rows, 0 exceptions.


In [11]:
short_char_chunks = pxt.create_view(
    'rag_ops_demo.short_char_chunks', docs,
    iterator=DocumentSplitter.create(
        document=docs.source_doc,
        separators='paragraph,char_limit',
        limit=72,
        overlap=0,
        metadata='title,heading,sourceline'
    )
)

Inserting rows into `short_char_chunks`: 1763 rows [00:00, 20191.74 rows/s]
Created view `short_char_chunks` with 1763 rows, 0 exceptions.


In [13]:
chunks.select(chunks.text, chunks.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .",{'1': 'Marc Chagall'}
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus) [1] Died 28 March 1985 (1985-03-28) (aged 97) Saint-Paul-de-Vence , France Nationality Russian, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Valentina (Vava) Brodsky ​ ​ ( m. 1952) ​ [3] Children 2 [4]",{'1': 'Marc Chagall'}
"Marc Chagall [a] (born Moishe Shagal ; 6 July [ O.S. 24 June] 1887 – 28 March 1985) was a Russian-French artist. [b] An early modernist , he was associated with the École de Paris as well as several major artistic styles and created works in a wide range of artistic formats, including painting, drawings, book illustrations, stained glass , stage sets, ceramics, tapestries and fine art prints.",{'1': 'Marc Chagall'}
"Chagall was born in 1887, into a Jewish family near Vitebsk , today in Belarus , but at that time in the Pale of Settlement of the Russian Empire. Before World War I , he travelled between Saint Petersburg , Paris , and Berlin . During that period, he created his own mixture and style of modern art, based on his ideas of Eastern European and Jewish folklore. He spent the wartime years in his native Belarus, becoming one of the country's most distinguished artists and a member of the modernist avant-garde , founding the Vitebsk Arts College . He later worked in and near Moscow in difficult conditions during hard times in Russia following the Bolshevik Revolution , before leaving again for Paris in 1923. During World War II , he escaped occupied France to the United States, where he lived in New York City for seven years before returning to France in 1948.",{'1': 'Marc Chagall'}
"Art critic Robert Hughes referred to Chagall as ""the quintessential Jewish artist of the twentieth century"". According to art historian Michael J. Lewis, Chagall was considered to be ""the last survivor of the first generation of European modernists"". For decades, he ""had also been respected as the world's pre-eminent Jewish artist"". [15] Using the medium of stained glass, he produced windows for the cathedrals of Reims and Metz as well as the Fraumünster in Zürich , windows for the UN and the Art Institute of Chicago and the Jerusalem Windows in Israel. He also did large-scale paintings, including part of the ceiling of the Paris Opéra . He experienced modernism's ""golden age"" in Paris, where ""he synthesized the art forms of Cubism , Symbolism , and Fauvism , and the influence of Fauvism gave rise to Surrealism "". Yet throughout these phases of his style ""he remained most emphatically a Jewish artist, whose work was one long dreamy reverie of life in his native village of Vitebsk."" [16] ""When Matisse dies"", Pablo Picasso remarked in the 1950s, ""Chagall will be the only painter left who understands

In [14]:
short_chunks.select(short_chunks.text, short_chunks.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡ,{'1': 'Marc Chagall'}
ортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한,{'1': 'Marc Chagall'}
국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lë,{'1': 'Marc Chagall'}
tzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemont,{'1': 'Marc Chagall'}
èis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi,{'1': 'Marc Chagall'}
"Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here",{'1': 'Marc Chagall'}
". For other uses, see Chagall (disambiguation) .",{'1': 'Marc Chagall'}
"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna , Vitebsk Governorate , Russian Empire (now Belarus) [1] Died 28 March 1985 (1985-03-",{'1': 'Marc Chagall'}
"28) (aged 97) Saint-Paul-de-Vence , France Nationality Russian, later French [2] Known for Painting stained glass Notable work See list of artworks by Marc Chagall Movement Cubism Expressionism School of Paris Spouses Bella Rosenfeld ​ ​ ( m. 1915; died 1944) ​ Valentina",{'1': 'Marc Chagall'}


In [15]:
short_char_chunks.select(short_char_chunks.text, short_char_chunks.heading).show(20)

text,heading
Marc Chagall - Wikipedia Jump to content Search Search,{}
Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտա,{'1': 'Marc Chagall'}
հայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (та,{'1': 'Marc Chagall'}
рашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά,{'1': 'Marc Chagall'}
Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrva,{'1': 'Marc Chagall'}
tski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswah,{'1': 'Marc Chagall'}
ili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy م,{'1': 'Marc Chagall'}
صرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbe,{'1': 'Marc Chagall'}
kcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Ro,{'1': 'Marc Chagall'}
mână Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina S,{'1': 'Marc Chagall'}


Now let's add a few more documents to our workflow. Notice how all of the downstream views are updated incrementally, processing just the new documents as they are inserted.

In [16]:
urls = [
    'https://en.wikipedia.org/wiki/Pierre-Auguste_Renoir',
    'https://en.wikipedia.org/wiki/Henri_Matisse',
    'https://en.wikipedia.org/wiki/Marcel_Duchamp'
]
docs.insert({'source_doc': url} for url in urls)

Inserting rows into `docs`: 3 rows [00:00, 3352.76 rows/s]
Inserting rows into `sentences`: 2099 rows [00:02, 737.95 rows/s]
Inserting rows into `chunks`: 275 rows [00:00, 21064.59 rows/s]
Inserting rows into `short_chunks`: 809 rows [00:00, 24956.18 rows/s]
Inserting rows into `short_char_chunks`: 2635 rows [00:00, 9001.13 rows/s]
Inserted 5821 rows with 0 errors.


UpdateStatus(num_rows=5821, num_computed_values=0, num_excs=0, updated_cols=[], cols_with_excs=[])

## Further Experiments

This is a good time to mention another important guiding principle of Pixeltable. The preceding examples all used the built-in `DocumentSplitter` class with various configurations. That's probably fine as a first cut or to prototype an application quickly, and it might be sufficient for some applications. But other applications might want to do more sophisticated kinds of chunking, implementing their own specialized logic or leveraging third-party tools. Pixeltable imposes no constraints on the AI or RAG operations a workflow uses: the iterator interface is highly general, and it's easy to implement new operations or adapt existing code or third-party tools into the Pixeltable workflow.

## Computing Embeddings

Next, let's look at how embedding indices can be added seamlessly to existing Pixeltable workflows. To compute our embeddings, we'll use the Huggingface `sentence_transformer` package, running it over the `chunks` view that broke our documents up into larger paragraphs. Pixeltable has a built-in `sentence_transformer` adapter, and all we have to do is add a new column that leverages it. Pixeltable takes care of the rest, applying the new column to all existing data in the view.

In [17]:
from pixeltable.functions.huggingface import sentence_transformer

chunks['minilm_embed'] =sentence_transformer(chunks.text, model_id='paraphrase-MiniLM-L6-v2')

Computing cells: 100%|███████████████████████████████████████| 480/480 [00:01<00:00, 452.39 cells/s]
Added 480 column values with 0 errors.


The new column is a *computed column*: it is defined as a function on top of existing data and updated incrementally as new data are added to the workflow. Let's have a look at how the new column affected the `chunks` view.

In [18]:
chunks

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
heading,json,
sourceline,int,
minilm_embed,"array((384,), dtype=FLOAT)","sentence_transformer(text, model_id='paraphrase-MiniLM-L6-v2')"
source_doc,document,


In [19]:
chunks.head()

pos,text,title,heading,sourceline,minilm_embed,source_doc
0,Marc Chagall - Wikipedia Jump to content Search Search,Marc Chagall - Wikipedia,{},0,"[-0.2623971, -0.11875597, -0.1327094, 0.04825155, 0.11987543, -0.0060522407, 0.32066354, 0.048358902, 0.29429528, -0.25400928, 0.6250124, 0.1780471, -0.0049093324, 0.26455668, -0.25857234, -0.04235539, 0.36528495, 0.23551314, 0.2759594, -0.37120554, 0.17573267, 0.0029874444, 0.23927839, -0.17485918, 0.19540693, -0.48891962, -0.7418267, -0.3072058, 0.23550892, -0.26159525, 0.5011105, -0.6918157, -0.2546256, -0.19169794, 0.52656704, 0.10684747, -0.35004127, -0.47890767, 0.27644458, 0.26248962, 0.055178195, -0.14610718, -0.21482663, 0.724693, -0.105297856, 0.5106915, -0.1817412, 0.4592062, 0.36721292, 0.5129047, -0.6575721, -0.44697943, -0.26266992, -0.4482015, 0.38971034, -0.16952093, -0.1143501, -0.0723624, 0.26450846, -0.57930374, 0.28047284, 0.09170102, -0.16436036, 0.10646355, -0.14615744, 0.19661124, 0.07363992, 0.21059889, 0.17992173, 0.15980835, -0.37105095, 0.25376248, -0.44671547, 0.657332, -0.3887278, -0.27646342, -0.7946083, -0.08837419, -0.27318326, -0.21008565, 0.57023406, -0.3434786, 0.15152001, -0.3031939, -0.043730196, -0.32914844, 0.25554967, 0.060874503, 0.23776944, 0.2961497, 0.18648748, -0.355997, 0.15256242, -0.084133826, -0.48064312, 0.30865636, 0.10970494, 0.11276869, 0.71860474, -0.21019451, ...]",/Users/asiegel/.pixeltable/file_cache/26f958802a424414b408646538f72715_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
1,"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .",Marc Chagall - Wikipedia,{'1': 'Marc Chagall'},820,"[-0.13631284, 0.40063256, -0.5300299, -0.1814317, -0.45316994, -0.12492252, 0.7796175, -0.24712454, 0.12325861, -0.22153562, 0.28455302, -0.43702734, 0.2315685, 0.14490259, -0.10851226, -0.28869808, 0.2181759, 0.7690574, -0.3198973, -0.4431276, 0.059898343, -0.08027286, 0.12561797, 0.20749918, -0.04779523, 0.24811892, 0.31219947, -0.08558024, -0.22011814, 0.35220224, 0.58977956, 0.07881065, 0.19522737, -0.14123651, 0.35313025, 0.248419, -0.13180904, 0.2722456, 0.28210595, 0.13397658, 0.10607683, -0.09232128, 0.09185548, 0.18311344, 0.12448602, -0.05098941, -0.17604476, 0.0156472, -0.05244754, -0.24283394, -0.42616746, -0.45576864, -0.35508007, -0.43146527, 0.30693352, -0.9420444, -0.1502328, 0.73327005, 0.0008648187, -0.3663265, -0.5619301, 0.21601504, -0.6227497, 0.43365508, -0.28584424, -0.33551326, -0.044290617, 0.34401017, -0.20072122, 0.051950324, -0.14124426, -0.12208992, 0.17209947, 0.24971412, -0.1148842, -0.3685053, 0.25038457, 0.22488211, -0.23860529, 0.014345817, 0.13038483, -0.0057234727, -0.030419111, -0.29217714, 0.23628002, -0.07539792, -0.011316142, -0.091147415, 0.65404445, -0.946898, 0.5174334, 0.2996515, 0.23870116, -0.06947715, 0.5011023, -0.08394453, 0.15647276, -0.33553213, 0.08633526, -0.110680446, ...]",/Users/asiegel/.pixeltable/file_cache/26f958802a424414b408646538f72715_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
2,"Marc Chagall Chagall, c. 1920 Born Moishe Shagal ( 1887-07-06 ) 6 July 1887 (N.S.) Liozna ,

Similarly, we might want to add a CLIP embedding to our workflow; once again, it's just another computed column:

In [20]:
from pixeltable.functions.huggingface import clip_text

chunks['clip_embed'] = clip_text(chunks.text, model_id='openai/clip-vit-base-patch32')

Computing cells: 100%|███████████████████████████████████████| 480/480 [00:02<00:00, 238.96 cells/s]
Added 480 column values with 0 errors.


In [21]:
chunks

Column Name,Type,Computed With
pos,int,
text,string,
title,string,
heading,json,
sourceline,int,
minilm_embed,"array((384,), dtype=FLOAT)","sentence_transformer(text, model_id='paraphrase-MiniLM-L6-v2')"
clip_embed,"array((512,), dtype=FLOAT)","clip_text(text, model_id='openai/clip-vit-base-patch32')"
source_doc,document,


In [22]:
chunks.head()

pos,text,title,heading,sourceline,minilm_embed,clip_embed,source_doc
0,Marc Chagall - Wikipedia Jump to content Search Search,Marc Chagall - Wikipedia,{},0,"[-0.2623971, -0.11875597, -0.1327094, 0.04825155, 0.11987543, -0.0060522407, 0.32066354, 0.048358902, 0.29429528, -0.25400928, 0.6250124, 0.1780471, -0.0049093324, 0.26455668, -0.25857234, -0.04235539, 0.36528495, 0.23551314, 0.2759594, -0.37120554, 0.17573267, 0.0029874444, 0.23927839, -0.17485918, 0.19540693, -0.48891962, -0.7418267, -0.3072058, 0.23550892, -0.26159525, 0.5011105, -0.6918157, -0.2546256, -0.19169794, 0.52656704, 0.10684747, -0.35004127, -0.47890767, 0.27644458, 0.26248962, 0.055178195, -0.14610718, -0.21482663, 0.724693, -0.105297856, 0.5106915, -0.1817412, 0.4592062, 0.36721292, 0.5129047, -0.6575721, -0.44697943, -0.26266992, -0.4482015, 0.38971034, -0.16952093, -0.1143501, -0.0723624, 0.26450846, -0.57930374, 0.28047284, 0.09170102, -0.16436036, 0.10646355, -0.14615744, 0.19661124, 0.07363992, 0.21059889, 0.17992173, 0.15980835, -0.37105095, 0.25376248, -0.44671547, 0.657332, -0.3887278, -0.27646342, -0.7946083, -0.08837419, -0.27318326, -0.21008565, 0.57023406, -0.3434786, 0.15152001, -0.3031939, -0.043730196, -0.32914844, 0.25554967, 0.060874503, 0.23776944, 0.2961497, 0.18648748, -0.355997, 0.15256242, -0.084133826, -0.48064312, 0.30865636, 0.10970494, 0.11276869, 0.71860474, -0.21019451, ...]","[0.43900785, -0.20442092, -0.3703841, -0.24842069, 0.2525649, 0.109884605, 0.17879923, -0.19681284, -0.21898562, 0.16312268, -0.11581852, 0.1092238, -0.39804864, 0.09145158, 0.102825396, -0.08496865, -0.26351961, 0.21744476, -0.30518532, -0.25689375, 0.3959956, 0.05023121, 0.06975499, 0.024919359, -0.23106097, 0.40560904, 0.05541487, 0.32912314, 0.30847397, -0.50972724, -0.09413456, -0.114752874, 0.06090342, -0.41492945, 0.22001785, -0.18334094, -0.1786417, -0.31954315, 0.2573876, -0.22376972, 0.18910305, 0.026809765, -0.004293302, 0.40239203, 0.20990945, 0.09746921, -0.130263, -0.22480457, -0.57115334, -0.08310897, 0.47325373, -0.18007915, 0.12506786, 0.086631425, 0.9078167, 0.2944164, 0.37376246, 0.0021954004, -0.45549917, 0.31297547, 0.15392725, 0.19201666, 0.40218055, 0.07109224, 0.1754787, -0.1596657, 0.23044428, 0.17929633, -0.062350426, -0.14394848, 0.083964534, -0.19787143, 0.15875755, 0.15464796, -0.12898104, -0.12406595, -0.45278224, 0.09551871, 0.002874703, 0.6825139, -0.5988117, -0.33829072, -0.34874046, 0.07570042, -0.35363954, -0.31604543, -0.1459959, -0.5174632, -0.13107206, 0.020408304, -0.4795367, -0.08089456, -1.0206141, -0.15478384, 0.23600452, 0.29316607, -0.44745418, -0.07385774, -0.11532665, -0.16539572, ...]",/Users/asiegel/.pixeltable/file_cache/26f958802a424414b408646538f72715_0_3c5b8031c7610e17a42ab6df79e614c2ee1a85dbd022497373d574cf65e15c51
1,"Marc Chagall 81 languages Afrikaans Alemannisch العربية Aragonés Արեւմտահայերէն Asturianu Azərbaycanca বাংলা Башҡортса Беларуская Беларуская (тарашкевіца) Български Català Čeština Cymraeg Dansk Deutsch Eesti Ελληνικά Español Esperanto Euskara فارسی Français Galego 한국어 Հայերեն हिन्दी Hrvatski Ido Bahasa Indonesia Interlingua Italiano עברית Jawa ქართული Kiswahili Latina Latviešu Lëtzebuergesch Lietuvių Magyar Македонски Malagasy مصرى Nederlands Nedersaksies 日本語 Norsk bokmål Norsk nynorsk Occitan Oʻzbekcha / ўзбекча پنجابی Picard Piemontèis Plattdüütsch Polski Português Română Runa Simi Русский Scots Shqip Sicilianu Simple English Slovenčina Slovenščina کوردی Српски / srpski Srpskohrvatski / српскохрватски Suomi Svenska ไทย Türkçe Українська Tiếng Việt Winaray 吴语 ייִדיש 粵語 中文 Edit links From Wikipedia, the free encyclopedia Russian-French artist (1887–1985) ""Chagall"" redirects here. For other uses, see Chagall (disambiguation) .",Marc Chagall - Wikipedia,{'1': 'Marc Chagall'},820,"[-0.13631284, 0.40063256, -0.5300299, -0.1814317, -0.45316994, -0.12492252, 0.7796175, -0.24712454, 0.12325861, -0.22153562, 0.28455302, -0.43702734, 0.2315685, 0.14490259, -0.10851226, -0.28869808, 0.2181759, 0.769